In [ ]:

import requests
import time

# ✅ Kakao REST API 키 입력 (직접 발급 필요)
KAKAO_API_KEY = "-"

# ✅ Kakao API
headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}

In [2]:
import pandas as pd

vol_df1= pd.read_csv("4_weekday.csv")
vol_df1

,spot_num,date,hour,inout,lane,volume
0,A-01,20250501,0,1,1,198
1,A-01,20250501,0,1,2,309
2,A-01,20250501,0,2,1,490
3,A-01,20250501,0,2,2,328
4,A-02,20250501,0,1,1,169
...,...,...,...,...,...,...
456898,F-09,20250530,23,2,2,239
456899,F-10,20250530,23,1,1,278
456900,F-10,20250530,23,1,2,236
456901,F-10,20250530,23,2,1,834


In [3]:
vol_df2= pd.read_csv("4_weekend.csv")
vol_df2

,spot_num,date,hour,inout,lane,volume
0,A-01,20250405,0,1,1,188
1,A-01,20250405,0,1,2,308
2,A-01,20250405,0,2,1,396
3,A-01,20250405,0,2,2,271
4,A-02,20250405,0,1,1,165
...,...,...,...,...,...,...
161022,F-09,20250427,23,2,2,121
161023,F-10,20250427,23,1,1,182
161024,F-10,20250427,23,1,2,173
161025,F-10,20250427,23,2,1,293


In [4]:
total_vol_df = pd.concat([vol_df1,vol_df2]).reset_index()
total_vol_df

,index,spot_num,date,hour,inout,lane,volume
0,0,A-01,20250501,0,1,1,198
1,1,A-01,20250501,0,1,2,309
2,2,A-01,20250501,0,2,1,490
3,3,A-01,20250501,0,2,2,328
4,4,A-02,20250501,0,1,1,169
...,...,...,...,...,...,...,...
617925,161022,F-09,20250427,23,2,2,121
617926,161023,F-10,20250427,23,1,1,182
617927,161024,F-10,20250427,23,1,2,173
617928,161025,F-10,20250427,23,2,1,293


In [5]:
spot_df= pd.read_csv("spot_with_address.csv")
spot_df

,spot_num,spot_name,x,y,lon,lat,구,동,지역
0,A-01,성산로(금화터널),195489.000000,452136.000000,126.946835,37.571502,서대문구,봉원동,북서부
1,A-02,사직로(사직터널),196756.776106,452546.638644,126.961184,37.575207,종로구,무악동,북서부
2,A-03,자하문로(자하문터널),197216.855046,454350.990432,126.966385,37.591465,종로구,청운동,북서부
3,A-04,대사관로(삼청터널),198648.893154,455200.108465,126.982598,37.599119,성북구,성북동,북서부
4,A-05,율곡로(안국역),198645.671347,452937.216603,126.982566,37.578731,종로구,안국동,북서부
...,...,...,...,...,...,...,...,...,...
134,F-06,경부고속도로,202107.000000,443264.000000,127.021724,37.491575,서초구,서초동,남동부
135,F-07,분당수서로,207716.000000,444241.000000,127.085158,37.500348,송파구,삼전동,남동부
136,F-08,강남순환로(관악터널),191832.000000,437667.000000,126.905601,37.441111,금천구,시흥동,남서부
137,F-09,서부간선지하도로,189706.000000,441044.000000,126.881529,37.471517,금천구,가산동,남서부


In [6]:
import pandas as pd

# ✅ 병합
merged_df = pd.merge(total_vol_df, spot_df, on="spot_num", how="left")

# ✅ 필요한 열 선택 및 결측치 제거 (선제적 정리)
# merged_df = merged_df[["spot_num", "spot_name", "date", "hour", "inout", "volume", "구", "동", "지역"]]
# merged_df.dropna(subset=["지역", "volume"], inplace=True)

# ✅ 지역별 혼잡도 집계 (유입, 유출 따로)
region_gb = (
    merged_df.groupby(["date", "hour","구", "동", "지역",  "spot_name",'lon',	'lat'])[["volume"]]
    .sum()
    .reset_index()
    .rename(columns={"volume": "총교통량"})
)

# # ✅ 유입/유출 해석 이름 붙이기
# region_gb["구분"] = region_gb["inout"].map({1: "유입", 2: "유출"})

# ✅ 시간대별 정렬
region_gb = region_gb.sort_values(by=["date", "hour", "지역"])

# ✅ 예시 저장
# region_gb.to_csv("지역별_시간별_교통량.csv", index=False, encoding="utf-8-sig")
# print("✅ 저장 완료: 지역별_시간별_교통량.csv")


In [7]:
region_gb

,date,hour,구,동,지역,spot_name,lon,lat,총교통량
0,20250405,0,강남구,삼성동,남동부,테헤란로(선릉역),127.050051,37.508634,1955
1,20250405,0,강남구,세곡동,남동부,밤고개로(세곡동사거리),127.105634,37.465363,332
2,20250405,0,강남구,세곡동,남동부,헌릉로(세곡동사거리),127.103285,37.467617,463
3,20250405,0,강남구,역삼동,남동부,언주로(매봉터널),127.045378,37.495175,937
5,20250405,0,강동구,천호동,남동부,광진교,127.113094,37.546933,190
...,...,...,...,...,...,...,...,...,...
94704,20250530,23,중구,봉래동1가,북서부,소월로(회현역),126.974277,37.559852,146
94705,20250530,23,중구,순화동,북서부,세종대로(서울역),126.972176,37.561329,1864
94706,20250530,23,중구,순화동,북서부,칠패로(숭례문),126.970502,37.562547,688
94710,20250530,23,중구,정동,북서부,서소문로(시청역),126.970693,37.565451,1462


In [8]:

# ✅ 데이터 불러오기 및 전처리 (동일)
merged_df = region_gb.copy()
merged_df["date"] = pd.to_datetime(merged_df["date"], format="%Y%m%d")
merged_df["weekday"] = merged_df["date"].dt.day_name()

day_map = {
    "Monday": "월요일",
    "Tuesday": "화요일",
    "Wednesday": "수요일",
    "Thursday": "목요일",
    "Friday": "금요일",
    "Saturday": "토요일",
    "Sunday": "일요일"
}
merged_df["weekday"] = merged_df["weekday"].map(day_map)


In [10]:
# ✅ 출근시간대(평일 7~9시)
commute_hours = [7,8,9]
weekdays_df = merged_df[merged_df["weekday"].isin(["월요일", "화요일", "수요일", "목요일", "금요일"])]
commute_df = weekdays_df[weekdays_df["hour"].isin(commute_hours)]

commute_top10 = commute_df.groupby("spot_name")["총교통량"].median().reset_index()
commute_top10 = commute_top10.sort_values("총교통량", ascending=False).head(10)
top10_commute_df_median = commute_top10.reset_index(drop=True)
top10_commute_df_median.index += 1

# 중앙 좌표까지 merge
# commute_top10 = commute_top10.merge(
#     merged_df.groupby("spot_name")[["lon","lat"]].median().reset_index(),
#     on="spot_name",
#     how="left"
# )

print("🚗 평일 출근시간대 top10")
display(top10_commute_df_median)

# ✅ 퇴근시간대(평일 16~18시)
leave_hours = [16,17,18]
leave_df = weekdays_df[weekdays_df["hour"].isin(leave_hours)]

leave_top10 = leave_df.groupby("spot_name")["총교통량"].median().reset_index()
leave_top10 = leave_top10.sort_values("총교통량", ascending=False).head(10)
top10_leave_df_median = leave_top10.reset_index(drop=True)
top10_leave_df_median.index += 1
# leave_top10 = leave_top10.merge(
#     merged_df.groupby("spot_name")[["lon","lat"]].median().reset_index(),
#     on="spot_name",
#     how="left"
# )

print("🚗 평일 퇴근시간대 top10")
display(top10_leave_df_median)


# ✅ 주말(토/일)

weekend_hours = [10,11,12,13,14,15,16,17]
weekend_df = merged_df[merged_df["weekday"].isin(["토요일", "일요일"])]
weekend_df = weekend_df[weekend_df["hour"].isin(weekend_hours)]

weekend_top10 = weekend_df.groupby("spot_name")["총교통량"].median().reset_index()
weekend_top10 = weekend_top10.sort_values("총교통량", ascending=False).head(10)
top10_weekend_median = weekend_top10.reset_index(drop=True)
top10_weekend_median.index += 1
# weekend_top10 = weekend_top10.merge(
#     merged_df.groupby("spot_name")[["lon","lat"]].median().reset_index(),
#     on="spot_name",
#     how="left"
# )

print("🚗 주말 top10")
display(top10_weekend_median)


🚗 평일 출근시간대 top10


,spot_name,총교통량
1,강변북로(난지한강공원),13112.0
2,올림픽대로,13063.0
3,강변북로,11949.5
4,올림픽대로(강일IC),11583.0
5,한남대교,10340.0
6,경부고속도로(양재IC),9653.0
7,경부고속도로,9578.5
8,양재대로(양재IC),8758.0
9,경인고속국도(신월IC),8171.0
10,성산대교,7942.0


🚗 평일 퇴근시간대 top10


,spot_name,총교통량
1,강변북로(난지한강공원),13989.0
2,올림픽대로(강일IC),11975.0
3,올림픽대로,11266.0
4,올림픽대로(개화IC),9596.5
5,경부고속도로(양재IC),9535.0
6,경인고속국도(신월IC),9273.5
7,양재대로(양재IC),9178.5
8,강변북로,9075.0
9,경부고속도로,8503.0
10,한남대교,8489.0


🚗 주말 top10


,spot_name,총교통량
1,강변북로(난지한강공원),13265.5
2,올림픽대로(강일IC),11854.5
3,올림픽대로,11752.0
4,강변북로,11595.5
5,경인고속국도(신월IC),9058.5
6,경부고속도로,9005.0
7,경부고속도로(양재IC),8999.0
8,올림픽대로(개화IC),8993.0
9,한남대교,8702.0
10,강변북로(구리시계),8060.5


In [11]:
df = merged_df

# ✅ 괄호 제거해서 도로명만 추출
df["도로명"] = df["spot_name"].str.replace(r"\(.*?\)", "", regex=True).str.strip()

# ✅ 평일만 필터링
weekdays_df = df[df["weekday"].isin(["월요일", "화요일", "수요일", "목요일", "금요일"])]

# ✅ 출근시간대
commute_hours = [7,8,9]
commute_df = weekdays_df[weekdays_df["hour"].isin(commute_hours)]

# ✅ 퇴근시간대
leave_hours = [16,17,18]
leave_df = weekdays_df[weekdays_df["hour"].isin(leave_hours)]



# ▶ 중앙값 기준 Top10 (도로명 기준)
commute_df_median = commute_df.groupby("도로명")["총교통량"].median().reset_index()
top10_commute_df_median = commute_df_median.sort_values("총교통량", ascending=False).head(10)
top10_commute_df_median = top10_commute_df_median.reset_index(drop=True)
top10_commute_df_median.index += 1

print("🚗 평일_출근시간대 도로별 총교통량 Top10 (도로명 기준)")
display(top10_commute_df_median)

leave_df_median = leave_df.groupby("도로명")["총교통량"].median().reset_index()
top10_leave_df_median = leave_df_median.sort_values("총교통량", ascending=False).head(10)
top10_leave_df_median = top10_leave_df_median.reset_index(drop=True)
top10_leave_df_median.index += 1

print("🚗 평일_퇴근시간대 도로별 총교통량 Top10 (도로명 기준)")
display(top10_leave_df_median)


weekend_hours = [10,11,12,13,14,15,16,17]
weekend_df = df[df["weekday"].isin(["토요일", "일요일"])]
weekend_df = weekend_df[weekend_df["hour"].isin(weekend_hours)]

weekend_median = weekend_df.groupby("도로명")["총교통량"].median().reset_index()
top10_weekend_median = weekend_median.sort_values("총교통량", ascending=False).head(10)
top10_weekend_median = top10_weekend_median.reset_index(drop=True)
top10_weekend_median.index += 1

print("🚗 주말(토/일) 도로별 총교통량 Top10 (도로명 기준)")
display(top10_weekend_median)


🚗 평일_출근시간대 도로별 총교통량 Top10 (도로명 기준)


,도로명,총교통량
1,강변북로,11849.0
2,올림픽대로,11331.0
3,한남대교,10340.0
4,경부고속도로,9641.0
5,양재대로,8758.0
6,경인고속국도,8171.0
7,성산대교,7942.0
8,내부순환로,7255.0
9,분당수서로,6952.0
10,마포대교,6733.0


🚗 평일_퇴근시간대 도로별 총교통량 Top10 (도로명 기준)


,도로명,총교통량
1,올림픽대로,11138.0
2,경인고속국도,9273.5
3,양재대로,9178.5
4,강변북로,9078.0
5,경부고속도로,9002.5
6,한남대교,8489.0
7,내부순환로,6978.5
8,분당수서로,6958.0
9,행주대교,6843.5
10,북부간선도로,6724.5


🚗 주말(토/일) 도로별 총교통량 Top10 (도로명 기준)


,도로명,총교통량
1,올림픽대로,11462.0
2,강변북로,11408.0
3,경인고속국도,9058.5
4,경부고속도로,9001.5
5,한남대교,8702.0
6,양재대로,7936.0
7,내부순환로,6953.5
8,분당수서로,6697.0
9,성산대교,6684.5
10,북부간선도로,6489.5


In [41]:
import pandas as pd
import requests
import folium
import time

# ✅ Kakao API
headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}

# ✅ Kakao 전국 장소 검색 함수
def kakao_search_places(keywords):
    results = []
    for keyword in keywords:
        try:
            res = requests.get(
                "https://dapi.kakao.com/v2/local/search/keyword.json",
                headers=headers,
                params={
                    "query": keyword,
                    "size": 1
                },
                timeout=5
            )
            print(f"🔎 [{keyword}] 응답 코드:", res.status_code)
            print(res.json())

            places = res.json().get("documents", [])
            if places:
                p = places[0]
                results.append({
                    "input_keyword": keyword,
                    "place_name": p["place_name"],
                    "address": p["road_address_name"] or p["address_name"],
                    "lon": float(p["x"]),
                    "lat": float(p["y"])
                })
            else:
                print(f"⚠️ '{keyword}' 전국 검색에서도 결과 없음")
        except Exception as e:
            print(f"⚠️ '{keyword}' 호출 오류:", e)
            time.sleep(1)
    return pd.DataFrame(results)

# ✅ 주말 여행지 키워드 (Kakao에서 잘 잡히는 형태)
keywords = [
    "스타필드하남", "롯데월드타워", "인천국제공항", "홍대입구역", "성수카페거리",
    "두물머리", "남이섬", "대부도", "인천조개구이", "속초해수욕장",
    "남양주카페거리", "광화문교보문고", "여의도IFC몰", "뚝섬유원지",
    "강릉중앙시장", "가평수상레저", "캐리비안베이", "홍천오션월드", "행리단길"
]

# ✅ Kakao API로 장소 데이터 수집
df_places = kakao_search_places(keywords)

# ✅ 결과 확인
print("\n✅ 수집된 주말 여행지 데이터")
print(df_places)

# ✅ CSV로 저장
df_places.to_csv("weekend_destinations_kakao_final.csv", index=False, encoding="utf-8-sig")

# ✅ folium 지도에 마커 표시
m = folium.Map(location=[37.5665, 126.9780], zoom_start=8)
for _, row in df_places.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=f"{row['place_name']}<br>{row['address']}",
    ).add_to(m)

# ✅ HTML 파일로 저장
m.save("weekend_destinations_map.html")
print("\n✅ 'weekend_destinations_map.html' 파일 생성 완료. 브라우저로 열어보세요!")


🔎 [스타필드하남] 응답 코드: 200
{'documents': [{'address_name': '경기 하남시 신장동 616', 'category_group_code': '', 'category_group_name': '', 'category_name': '가정,생활 > 복합쇼핑몰', 'distance': '', 'id': '25894201', 'phone': '1833-9001', 'place_name': '스타필드 하남', 'place_url': 'http://place.map.kakao.com/25894201', 'road_address_name': '경기 하남시 미사대로 750', 'x': '127.22367994072209', 'y': '37.545512916608395'}], 'meta': {'is_end': False, 'pageable_count': 45, 'same_name': {'keyword': '스타필드하남', 'region': [], 'selected_region': ''}, 'total_count': 645}}
🔎 [롯데월드타워] 응답 코드: 200
{'documents': [{'address_name': '서울 송파구 신천동 29', 'category_group_code': '', 'category_group_name': '', 'category_name': '부동산 > 빌딩', 'distance': '', 'id': '23444676', 'phone': '02-3213-5000', 'place_name': '롯데월드타워', 'place_url': 'http://place.map.kakao.com/23444676', 'road_address_name': '서울 송파구 올림픽로 300', 'x': '127.10255558658325', 'y': '37.51260447840551'}], 'meta': {'is_end': False, 'pageable_count': 45, 'same_name': {'keyword': '롯데월드타워', 'r

In [54]:
df_places

,input_keyword,place_name,address,lon,lat
0,스타필드하남,스타필드 하남,경기 하남시 미사대로 750,127.223680,37.545513
1,롯데월드타워,롯데월드타워,서울 송파구 올림픽로 300,127.102556,37.512604
2,인천국제공항,인천국제공항,인천 중구 공항로 272,126.450920,37.449168
3,홍대입구역,홍대입구역 2호선,서울 마포구 양화로 지하 160,126.923779,37.556871
4,성수카페거리,서울숲카페거리,서울 성동구 성수동1가 668-11,127.043029,37.546444
5,두물머리,두물머리,경기 양평군 양서면 양수리,127.315830,37.532813
6,남이섬,남이섬,강원특별자치도 춘천시 남산면 남이섬길 1,127.525210,37.791441
7,대부도,대부도,경기 안산시 단원구 대부동동,126.596768,37.246139
8,인천조개구이,서해안조개광장,인천 중구 마시란로 65,126.418714,37.428011
9,속초해수욕장,속초해수욕장,강원특별자치도 속초시 해오름로 186,128.604098,38.190053


In [55]:
import pandas as pd
import numpy as np
import requests
import folium
import random
import time
from math import radians, cos, sin, asin, sqrt

np.random.seed(20250744)
random.seed(20250744)

# ✅ Haversine
def haversine(lon1, lat1, lon2, lat2):
    R = 6371000
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return R * c


# ✅ Kakao에서 근처 아파트 이름, 주소, 그리고 그 주소를 다시 위경도로
def get_nearby_apartment_location(lon, lat):
    brands = ["래미안","자이","힐스테이트","푸르지오","롯데캐슬","더샵","e편한세상",
              "아이파크","센트레빌","두산위브","휴먼시아","주공","아파트","빌라","주택"]
    # 제외할 키워드들
    exclude_keywords = ["전시관", "예정", "분양", "견본", "모델", "상가", "사무실", "오피스텔", "업무시설", "프라자"]
    
    for radius in [1000,2000,3000]:
        for keyword in brands:
            try:
                res = requests.get(
                    "https://dapi.kakao.com/v2/local/search/keyword.json",
                    headers=headers,
                    params={"query": keyword, "x": lon, "y": lat, "radius": radius, "size": 10},
                    timeout=5
                )
                places = res.json().get("documents", [])
                # 주거용 건물만 필터링하고 제외 키워드 포함된 것들 제거
                places = [p for p in places if 
                         any(b in p["place_name"] for b in ["아파트","빌라","주택"]) and
                         not any(ex in p["place_name"] for ex in exclude_keywords)]
                if places:
                    p = random.choice(places)
                    # place_name, address_name, 좌표 반환
                    return p["place_name"], p["road_address_name"] or p["address_name"], float(p["x"]), float(p["y"])
            except:
                time.sleep(1)
    # 실패하면 임의 위치
    return "근처주택", "주소없음", lon, lat

# ✅ 경로 생성 - 도착지 분포 개선 + 출발지 다양성 확보
def make_weekend_travel_routes_between_within_line(way_row, destinations, homes_df,
                                                   count=5, min_distance_m=30000, line_tolerance=2000):
    results, used_pairs, attempts = [], set(), 0
    used_destinations = set()  # 사용된 도착지 추적
    used_home_addresses = set()  # 사용된 출발지 주소 추적
    
    while len(results) < count and attempts < 10000:
        attempts += 1
        home_row = homes_df.sample(1).iloc[0]
        
        # 도착지 선택 로직 개선
        available_destinations = destinations[~destinations.index.isin(used_destinations)]
        
        # 사용 가능한 도착지가 없으면 초기화 (재사용 허용)
        if len(available_destinations) == 0:
            available_destinations = destinations
            used_destinations = set()
        
        dest_row = available_destinations.sample(1).iloc[0]

        # 출발-도착 거리가 너무 짧으면 제외
        direct_dist = haversine(home_row["lon"], home_row["lat"], dest_row["lon"], dest_row["lat"])
        if direct_dist < min_distance_m:
            continue
        
        # 출발-경유 + 경유-도착 거리
        path_via_way = ( 
            haversine(home_row["lon"], home_row["lat"], way_row["lon"], way_row["lat"]) +
            haversine(way_row["lon"], way_row["lat"], dest_row["lon"], dest_row["lat"])
        )
        
        # 경유지가 직선상에 있는지 (tolerance 내)
        if abs(path_via_way - direct_dist) > line_tolerance:
            continue

        pair_key = (home_row.name, dest_row.name)
        if pair_key in used_pairs:
            continue

        # ✅ 출발지 아파트 정보 가져오기
        home_name, home_addr, home_lon, home_lat = get_nearby_apartment_location(home_row["lon"], home_row["lat"])
        
        # ✅ 같은 주소의 아파트 중복 방지
        if home_addr in used_home_addresses:
            continue
        
        results.append({
            "type": "주말여행",
            "출발_건물명": home_name,
            "출발_주소": home_addr,
            "출발_lon": home_lon,  # 수정된 좌표 사용
            "출발_lat": home_lat,  # 수정된 좌표 사용
            "경유_도로명": way_row["spot_name"],
            "경유_lon": way_row["lon"], "경유_lat": way_row["lat"],
            "도착_장소명": dest_row["place_name"],
            "도착_주소": dest_row["address"],
            "도착_lon": dest_row["lon"], "도착_lat": dest_row["lat"]
        })
        used_pairs.add(pair_key)
        used_destinations.add(dest_row.name)  # 사용된 도착지 추가
        used_home_addresses.add(home_addr)  # 사용된 출발지 주소 추가
        
    return pd.DataFrame(results)

# ✅ 전체 경로 생성 - 글로벌 도착지 분포 관리
def generate_diverse_weekend_routes(weekend_top10, destinations, homes_df):
    all_results = []
    global_used_destinations = set()
    
    for _, way_row in weekend_top10.iterrows():
        way_series = pd.Series({
            "spot_name": way_row["spot_name"], 
            "lon": way_row["lon"], 
            "lat": way_row["lat"]
        })
        
        # 각 경유지마다 사용 가능한 도착지 필터링
        available_destinations = destinations[~destinations.index.isin(global_used_destinations)]
        
        # 사용 가능한 도착지가 2개 미만이면 일부 재사용 허용
        if len(available_destinations) < 2:
            # 가장 적게 사용된 도착지들을 다시 사용 가능하게 만들기
            destination_counts = {}
            for result in all_results:
                dest_name = result["도착_장소명"]
                destination_counts[dest_name] = destination_counts.get(dest_name, 0) + 1
            
            # 사용 횟수가 적은 순으로 정렬하여 일부 재사용 허용
            if destination_counts:
                min_count = min(destination_counts.values())
                reusable_destinations = [k for k, v in destination_counts.items() if v == min_count]
                # 재사용 가능한 도착지의 인덱스 찾기
                reusable_indices = destinations[destinations["place_name"].isin(reusable_destinations)].index
                global_used_destinations -= set(reusable_indices)
                available_destinations = destinations[~destinations.index.isin(global_used_destinations)]
        
        # 경로 생성
        routes = make_weekend_travel_routes_between_within_line(
            way_series, available_destinations, homes_df, 
            count=5, min_distance_m=3000, line_tolerance=2000
        )
        
        if len(routes) > 0:
            all_results.extend(routes.to_dict('records'))
            # 사용된 도착지 업데이트
            for route in routes.to_dict('records'):
                dest_indices = destinations[destinations["place_name"] == route["도착_장소명"]].index
                global_used_destinations.update(dest_indices)
    
    return pd.DataFrame(all_results)

# ✅ 데이터 불러오기 예시
# merged_df = pd.read_csv("your_traffic_data.csv")
df_places = pd.read_csv("weekend_destinations_kakao_final.csv")

# ✅ 주말 top10 도로
weekend_top10 = (
    merged_df[merged_df["weekday"].isin(["토요일","일요일"])]
    .groupby("spot_name")["총교통량"].median()
    .reset_index()
    .sort_values("총교통량", ascending=False)
    .head(10)
    .merge(merged_df.groupby("spot_name")[["lon","lat"]].median().reset_index(), on="spot_name")
)

# ✅ 출발 후보지 - 서울 전체에서 고르게 샘플링 (개선된 버전)
def create_diverse_home_candidates(merged_df, sample_size=200):
    """서울 전체에서 지역별로 고르게 출발 후보지를 샘플링"""
    all_candidates = merged_df.drop_duplicates(subset=["spot_name"])[["spot_name","lon","lat"]]
    
    # 서울 대략적 경계 (위경도 기준) - 더 넓게 설정
    seoul_bounds = {
        'lat_min': 37.42, 'lat_max': 37.72,
        'lon_min': 126.72, 'lon_max': 127.25
    }
    
    # 서울 내 후보지만 필터링
    seoul_candidates = all_candidates[
        (all_candidates['lat'] >= seoul_bounds['lat_min']) & 
        (all_candidates['lat'] <= seoul_bounds['lat_max']) &
        (all_candidates['lon'] >= seoul_bounds['lon_min']) & 
        (all_candidates['lon'] <= seoul_bounds['lon_max'])
    ].copy()
    
    # 서울 중심점 (시청 기준)
    center_lat, center_lon = 37.5665, 126.9780
    
    # 중심점으로부터의 거리 계산
    seoul_candidates['distance_from_center'] = seoul_candidates.apply(
        lambda row: haversine(center_lon, center_lat, row['lon'], row['lat']), axis=1
    )
    
    # 거리별로 구간 나누기 (동심원 형태)
    distance_bins = [0, 5000, 10000, 15000, 20000, 30000]  # 5km, 10km, 15km, 20km, 30km
    seoul_candidates['distance_bin'] = pd.cut(seoul_candidates['distance_from_center'], 
                                            bins=distance_bins, labels=False, include_lowest=True)
    
    # 방향별로 나누기 (8방향: 북, 북동, 동, 남동, 남, 남서, 서, 북서)
    def get_direction(lat, lon):
        # 중심점 대비 각도 계산
        import math
        dlat = lat - center_lat
        dlon = lon - center_lon
        angle = math.atan2(dlat, dlon) * 180 / math.pi
        
        # 8방향으로 분류
        if angle >= -22.5 and angle < 22.5:
            return '동'
        elif angle >= 22.5 and angle < 67.5:
            return '북동'
        elif angle >= 67.5 and angle < 112.5:
            return '북'
        elif angle >= 112.5 and angle < 157.5:
            return '북서'
        elif angle >= 157.5 or angle < -157.5:
            return '서'
        elif angle >= -157.5 and angle < -112.5:
            return '남서'
        elif angle >= -112.5 and angle < -67.5:
            return '남'
        else:  # -67.5 to -22.5
            return '남동'
    
    seoul_candidates['direction'] = seoul_candidates.apply(
        lambda row: get_direction(row['lat'], row['lon']), axis=1
    )
    
    # 각 (거리구간 × 방향) 조합에서 균등하게 샘플링
    sampled_homes = []
    unique_distance_bins = seoul_candidates['distance_bin'].dropna().unique()
    unique_directions = seoul_candidates['direction'].unique()
    
    samples_per_group = max(1, sample_size // (len(unique_distance_bins) * len(unique_directions)))
    
    for dist_bin in unique_distance_bins:
        for direction in unique_directions:
            group_candidates = seoul_candidates[
                (seoul_candidates['distance_bin'] == dist_bin) &
                (seoul_candidates['direction'] == direction)
            ]
            if len(group_candidates) > 0:
                n_samples = min(samples_per_group, len(group_candidates))
                sampled = group_candidates.sample(n_samples, random_state=20250744+int(dist_bin)*8+hash(direction)%100)
                sampled_homes.append(sampled)
    
    if sampled_homes:
        result = pd.concat(sampled_homes, ignore_index=True)
        
        # 목표 샘플 수에 맞추기
        if len(result) > sample_size:
            result = result.sample(sample_size, random_state=20250744)
        elif len(result) < sample_size:
            # 부족하면 전체에서 추가 샘플링 (기존 샘플과 중복되지 않게)
            additional_needed = sample_size - len(result)
            remaining = seoul_candidates[~seoul_candidates.index.isin(result.index)]
            if len(remaining) > 0:
                additional = remaining.sample(min(additional_needed, len(remaining)), random_state=20250744)
                result = pd.concat([result, additional], ignore_index=True)
        
        return result[['spot_name', 'lon', 'lat']]
    else:
        # fallback: 기존 방식
        return seoul_candidates.sample(min(sample_size, len(seoul_candidates)), random_state=20250744)[['spot_name', 'lon', 'lat']]

# ✅ 출발 후보지 생성
sampled_homes = create_diverse_home_candidates(merged_df, sample_size=200)

# ✅ 개선된 경로 생성
weekend_routes = generate_diverse_weekend_routes(weekend_top10, df_places, sampled_homes)

# ✅ 도착지 분포 확인
print("도착지별 사용 횟수:")
print(weekend_routes["도착_장소명"].value_counts())
print(f"\n총 고유 도착지 수: {weekend_routes['도착_장소명'].nunique()}")
print(f"총 경로 수: {len(weekend_routes)}")

weekend_routes

도착지별 사용 횟수:
도착_장소명
별내동 카페거리             3
서해안조개광장              3
속초해수욕장               3
남이섬                  3
가평빠지 캠프통포레스트 수상레저    3
두물머리                 3
스타필드 하남              3
대부도                  3
IFC몰                 3
인천국제공항               3
강릉중앙시장               3
비발디파크 오션월드           3
롯데월드타워               2
행리단길                 2
뚝섬한강공원               2
서울숲카페거리              2
교보문고 광화문점            2
캐리비안베이               2
홍대입구역 2호선            2
Name: count, dtype: int64

총 고유 도착지 수: 19
총 경로 수: 50


,type,출발_건물명,출발_주소,출발_lon,출발_lat,경유_도로명,경유_lon,경유_lat,도착_장소명,도착_주소,도착_lon,도착_lat
0,주말여행,DMC자이더리버아파트 배드민턴장,경기 고양시 덕양구 대덕산로 90,126.869254,37.579710,강변북로(난지한강공원),126.870423,37.573624,별내동 카페거리,경기 남양주시 별내동 946-7,127.127001,37.658012
1,주말여행,운현신화타워아파트,서울 종로구 삼일대로32길 36,126.989137,37.575102,강변북로(난지한강공원),126.870423,37.573624,서해안조개광장,인천 중구 마시란로 65,126.418714,37.428011
2,주말여행,화곡푸르지오아파트 관리사무소,서울 강서구 화곡로13길 107,126.831185,37.544099,강변북로(난지한강공원),126.870423,37.573624,비발디파크 오션월드,강원특별자치도 홍천군 서면 한치골길 262,127.686912,37.649953
3,주말여행,단독주택(개화2) 전기차충전소,서울 강서구 개화2길 3-7,126.798876,37.583661,강변북로(난지한강공원),126.870423,37.573624,강릉중앙시장,강원특별자치도 강릉시 금성로 21,128.898624,37.754025
4,주말여행,양원역금호어울림포레스트아파트,서울 중랑구 양원역로10길 75,127.110014,37.607757,강변북로(난지한강공원),126.870423,37.573624,인천국제공항,인천 중구 공항로 272,126.450920,37.449168
5,주말여행,나인원한남아파트,서울 용산구 한남대로 91,127.003708,37.536356,올림픽대로,126.971274,37.508909,IFC몰,서울 영등포구 국제금융로 10,126.925896,37.525042
6,주말여행,목동삼성아파트,서울 양천구 신목로 7,126.876372,37.515767,올림픽대로,126.971274,37.508909,서울숲카페거리,서울 성동구 성수동1가 668-11,127.043029,37.546444
7,주말여행,중림동삼성사이버빌리지아파트(201),서울 중구 중림로4길 41,126.964666,37.557218,올림픽대로,126.971274,37.508909,행리단길,경기 수원시 팔달구 신풍동 27-2,127.014128,37.285457
8,주말여행,철산자이더헤리티지아파트,경기 광명시 시청로 70,126.869033,37.482077,올림픽대로,126.971274,37.508909,롯데월드타워,서울 송파구 올림픽로 300,127.102556,37.512604
9,주말여행,래미안원베일리아파트 GATE2,서울 서초구 반포동 1-1,126.996460,37.506576,올림픽대로,126.971274,37.508909,홍대입구역 2호선,서울 마포구 양화로 지하 160,126.923779,37.556871


In [58]:
weekend_routes.to_csv('./weekend_route_maps/최종_주말_도로.csv')

In [ ]:
import matplotlib.pyplot as plt

# ✅ 경유 도로별 색상
unique_ways = weekend_routes["경유_도로명"].unique()
colors = plt.get_cmap("tab20").colors

way_color_map = {way: f'#{int(r*255):02X}{int(g*255):02X}{int(b*255):02X}' 
                 for way, (r,g,b) in zip(unique_ways, colors)}

# ✅ folium 지도
m_weekend = folium.Map(location=[37.5665, 126.9780], zoom_start=8)

for idx, row in weekend_routes.iterrows():
    line_color = way_color_map.get(row["경유_도로명"], 'gray')
    
    # ✅ 출발 → 경유 (실선)
    folium.PolyLine(
        locations=[[row["출발_lat"], row["출발_lon"]],
                   [row["경유_lat"], row["경유_lon"]]],
        color=line_color, weight=3, opacity=0.8
    ).add_to(m_weekend)
    
    # ✅ 경유 → 도착 (점선)
    folium.PolyLine(
        locations=[[row["경유_lat"], row["경유_lon"]],
                   [row["도착_lat"], row["도착_lon"]]],
        color=line_color, weight=3, opacity=0.8,
        dash_array='5, 10'  # 선 길이 5, 간격 10
    ).add_to(m_weekend)
    
    # ✅ 마커
    folium.Marker(
        location=[row["출발_lat"], row["출발_lon"]],
        popup=f"출발: {row['출발_건물명']}",
        icon=folium.Icon(color='red', icon='play', prefix='fa')
    ).add_to(m_weekend)
    
    folium.Marker(
        location=[row["경유_lat"], row["경유_lon"]],
        popup=f"경유: {row['경유_도로명']}",
        icon=folium.Icon(color='orange', icon='road', prefix='fa')
    ).add_to(m_weekend)
    
    folium.Marker(
        location=[row["도착_lat"], row["도착_lon"]],
        popup=f"도착: {row['도착_장소명']}",
        icon=folium.Icon(color='blue', icon='flag', prefix='fa')
    ).add_to(m_weekend)

# m_weekend.save("../주말_여행_경로_지도_최종.html")
# print("\n✅ '주말_여행_경로_지도_최종.html' 생성 완료! 브라우저에서 열어보세요.")
m_weekend

In [57]:
import matplotlib.pyplot as plt
import folium
import os

# ✅ 지도 저장용 폴더 생성
output_folder = "weekend_route_maps"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# ✅ 경유지별로 그룹핑
waypoint_groups = weekend_routes.groupby("경유_도로명")

# ✅ 색상 팔레트 (더 많은 색상)
def generate_colors(n):
    """n개의 서로 다른 색상 생성"""
    if n <= 20:
        colors = plt.get_cmap("tab20").colors
    else:
        colors = plt.get_cmap("hsv", n).colors
    
    return [f'#{int(r*255):02X}{int(g*255):02X}{int(b*255):02X}' 
            for r, g, b, *_ in colors[:n]]

# ✅ 경유지별로 개별 지도 생성
for waypoint_name, routes_group in waypoint_groups:
    print(f"\n🗺️ '{waypoint_name}' 경유지 지도 생성 중...")
    
    # 해당 경유지의 좌표를 중심으로 지도 생성
    waypoint_lat = routes_group.iloc[0]["경유_lat"]
    waypoint_lon = routes_group.iloc[0]["경유_lon"]
    
    # 지도 생성
    m = folium.Map(location=[waypoint_lat, waypoint_lon], zoom_start=10)
    
    # 경로별로 다른 색상 할당
    route_colors = generate_colors(len(routes_group))
    
    # 각 경로별로 처리
    for idx, (_, route) in enumerate(routes_group.iterrows()):
        route_color = route_colors[idx % len(route_colors)]
        
        # ✅ 출발 → 경유 (실선)
        folium.PolyLine(
            locations=[[route["출발_lat"], route["출발_lon"]],
                       [route["경유_lat"], route["경유_lon"]]],
            color=route_color, 
            weight=4, 
            opacity=0.8,
            popup=f"출발: {route['출발_건물명']} → 경유: {route['경유_도로명']}"
        ).add_to(m)
        
        # ✅ 경유 → 도착 (점선)
        folium.PolyLine(
            locations=[[route["경유_lat"], route["경유_lon"]],
                       [route["도착_lat"], route["도착_lon"]]],
            color=route_color, 
            weight=4, 
            opacity=0.8,
            dash_array='8, 12',  # 점선 패턴
            popup=f"경유: {route['경유_도로명']} → 도착: {route['도착_장소명']}"
        ).add_to(m)
        
        # ✅ 출발지 마커 (빨간색)
        folium.Marker(
            location=[route["출발_lat"], route["출발_lon"]],
            popup=folium.Popup(
                f"""
                <b>🏠 출발지</b><br>
                건물명: {route['출발_건물명']}<br>
                주소: {route['출발_주소']}<br>
                경로 색상: <span style='color:{route_color}'>●</span>
                """, 
                max_width=300
            ),
            icon=folium.Icon(color='red', icon='home', prefix='fa'),
            tooltip=f"출발: {route['출발_건물명']}"
        ).add_to(m)
        
        # ✅ 도착지 마커 (파란색)
        folium.Marker(
            location=[route["도착_lat"], route["도착_lon"]],
            popup=folium.Popup(
                f"""
                <b>🏁 도착지</b><br>
                장소명: {route['도착_장소명']}<br>
                주소: {route['도착_주소']}<br>
                경로 색상: <span style='color:{route_color}'>●</span>
                """, 
                max_width=300
            ),
            icon=folium.Icon(color='blue', icon='flag-checkered', prefix='fa'),
            tooltip=f"도착: {route['도착_장소명']}"
        ).add_to(m)
    
    # ✅ 경유지 마커 (주황색, 크게)
    folium.Marker(
        location=[waypoint_lat, waypoint_lon],
        popup=folium.Popup(
            f"""
            <b>🛣️ 경유지</b><br>
            도로명: {waypoint_name}<br>
            총 {len(routes_group)}개 경로 경유<br>
            <small>주말 교통량이 많은 도로</small>
            """, 
            max_width=300
        ),
        icon=folium.Icon(color='orange', icon='road', prefix='fa', icon_size=(20, 20)),
        tooltip=f"경유지: {waypoint_name}"
    ).add_to(m)
    
    # ✅ 범례 추가
    legend_html = f'''
    <div style="position: fixed; 
                top: 10px; right: 10px; width: 200px; height: auto; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:14px; padding: 10px">
    <h4>{waypoint_name}</h4>
    <p><i class="fa fa-home" style="color:red"></i> 출발지 ({len(routes_group)}개)</p>
    <p><i class="fa fa-road" style="color:orange"></i> 경유지</p>
    <p><i class="fa fa-flag-checkered" style="color:blue"></i> 도착지 ({len(routes_group)}개)</p>
    <hr>
    <p><strong>실선:</strong> 출발→경유</p>
    <p><strong>점선:</strong> 경유→도착</p>
    <p><small>각 경로별로 다른 색상</small></p>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))
    
    # ✅ 파일명 안전하게 처리 (특수문자 제거)
    safe_filename = "".join(c for c in waypoint_name if c.isalnum() or c in (' ', '-', '_')).rstrip()
    safe_filename = safe_filename.replace(' ', '_')
    
    # ✅ 지도 저장
    map_filename = f"{output_folder}/{safe_filename}_경유지_지도.html"
    m.save(map_filename)
    print(f"   ✅ 저장 완료: {map_filename}")

# ✅ 전체 경로 통합 지도도 생성
print(f"\n🗺️ 전체 경로 통합 지도 생성 중...")

# 전체 지도
m_all = folium.Map(location=[37.5665, 126.9780], zoom_start=9)

# 경유지별 색상 (경유지마다 다른 색상)
unique_waypoints = weekend_routes["경유_도로명"].unique()
waypoint_colors = generate_colors(len(unique_waypoints))
waypoint_color_map = {waypoint: color for waypoint, color in zip(unique_waypoints, waypoint_colors)}

# 모든 경로 표시
for idx, route in weekend_routes.iterrows():
    waypoint_color = waypoint_color_map[route["경유_도로명"]]
    
    # 출발 → 경유 (실선)
    folium.PolyLine(
        locations=[[route["출발_lat"], route["출발_lon"]],
                   [route["경유_lat"], route["경유_lon"]]],
        color=waypoint_color, 
        weight=2, 
        opacity=0.7
    ).add_to(m_all)
    
    # 경유 → 도착 (점선)
    folium.PolyLine(
        locations=[[route["경유_lat"], route["경유_lon"]],
                   [route["도착_lat"], route["도착_lon"]]],
        color=waypoint_color, 
        weight=2, 
        opacity=0.7,
        dash_array='5, 8'
    ).add_to(m_all)

# 경유지만 마커로 표시 (전체 지도에서는 너무 많으므로)
for waypoint, color in waypoint_color_map.items():
    waypoint_data = weekend_routes[weekend_routes["경유_도로명"] == waypoint].iloc[0]
    
    folium.Marker(
        location=[waypoint_data["경유_lat"], waypoint_data["경유_lon"]],
        popup=f"경유지: {waypoint}",
        icon=folium.Icon(color='orange', icon='road', prefix='fa'),
        tooltip=waypoint
    ).add_to(m_all)

# 전체 지도 저장
all_map_filename = f"{output_folder}/전체_주말여행경로_통합지도.html"
m_all.save(all_map_filename)
print(f"   ✅ 저장 완료: {all_map_filename}")

print(f"\n🎉 총 {len(unique_waypoints)+1}개 지도 파일이 '{output_folder}' 폴더에 생성되었습니다!")
print(f"   📁 개별 경유지 지도: {len(unique_waypoints)}개")
print(f"   📁 전체 통합 지도: 1개")

# ✅ 생성된 파일 목록 출력
print(f"\n📋 생성된 파일 목록:")
for waypoint in unique_waypoints:
    safe_filename = "".join(c for c in waypoint if c.isalnum() or c in (' ', '-', '_')).rstrip().replace(' ', '_')
    print(f"   • {safe_filename}_경유지_지도.html")
print(f"   • 전체_주말여행경로_통합지도.html")


🗺️ '강변북로' 경유지 지도 생성 중...
   ✅ 저장 완료: weekend_route_maps/강변북로_경유지_지도.html

🗺️ '강변북로(난지한강공원)' 경유지 지도 생성 중...
   ✅ 저장 완료: weekend_route_maps/강변북로난지한강공원_경유지_지도.html

🗺️ '경부고속도로' 경유지 지도 생성 중...
   ✅ 저장 완료: weekend_route_maps/경부고속도로_경유지_지도.html

🗺️ '경부고속도로(양재IC)' 경유지 지도 생성 중...
   ✅ 저장 완료: weekend_route_maps/경부고속도로양재IC_경유지_지도.html

🗺️ '경인고속국도(신월IC)' 경유지 지도 생성 중...
   ✅ 저장 완료: weekend_route_maps/경인고속국도신월IC_경유지_지도.html

🗺️ '내부순환로' 경유지 지도 생성 중...
   ✅ 저장 완료: weekend_route_maps/내부순환로_경유지_지도.html

🗺️ '올림픽대로' 경유지 지도 생성 중...
   ✅ 저장 완료: weekend_route_maps/올림픽대로_경유지_지도.html

🗺️ '올림픽대로(강일IC)' 경유지 지도 생성 중...
   ✅ 저장 완료: weekend_route_maps/올림픽대로강일IC_경유지_지도.html

🗺️ '올림픽대로(개화IC)' 경유지 지도 생성 중...
   ✅ 저장 완료: weekend_route_maps/올림픽대로개화IC_경유지_지도.html

🗺️ '한남대교' 경유지 지도 생성 중...
   ✅ 저장 완료: weekend_route_maps/한남대교_경유지_지도.html

🗺️ 전체 경로 통합 지도 생성 중...
   ✅ 저장 완료: weekend_route_maps/전체_주말여행경로_통합지도.html

🎉 총 11개 지도 파일이 'weekend_route_maps' 폴더에 생성되었습니다!
   📁 개별 경유지 지도: 10개
   📁 전체 통합 지도: 1개

📋 생성된 파일 목록:
   • 강변북로

In [60]:
# UTF-8 인코딩으로 저장 (권장)
weekend_routes.to_csv('./weekend_route_maps/최종_주말_도로.csv', 
                     encoding='utf-8-sig', 
                     index=False)